In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import ArrayLike
from typing import Union

import os
os.chdir('..')  # This changes the working directory to DiffGFDN

In [ ]:
def gfdn_flops(N: ArrayLike, B: int, is_parallel:bool=False):
    if is_parallel:
        return B*(2*np.power(N, 2) + 4*N + 1)
    else:
       return 2*np.power(N, 2) + N + 27*N*B + 1

def mlp_flops(num_layers: Union[int,ArrayLike], num_neurons: Union[int, ArrayLike], F:int) -> ArrayLike:
    return num_layers * (2*np.power(num_neurons,2) + num_neurons) + F*(2*num_neurons + 1)
    

In [ ]:
B = 8
N = np.arange(6, 24, 4)
flops_single = gfdn_flops(N, B)
flops_parallel = gfdn_flops(N, B, is_parallel=True)
plt.figure()
plt.plot(N, np.vstack((flops_single, flops_parallel)).T, '-x')
plt.xlabel('Number of delay lines')
plt.ylabel('FLOPS per sample')
plt.legend(['Single frequency-dep GFDN', 'Sum of parallel GFDNs'])
plt.savefig('figures/compare_flops_subband_GFDNs.png')
plt.show()

In [ ]:
num_layers = np.arange(1, 10, 1)
num_neurons = np.power(2, np.arange(5, 10))
NL, NN = np.meshgrid(num_layers, num_neurons)
num_groups = 2
B = 8
flops_mlp_single = mlp_flops(NL, NN, 4 * num_groups * B)
flops_mlp_parallel = B * mlp_flops(NL, NN, 2*num_groups)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(NL, NN, np.log10(flops_mlp_single))
ax.plot_surface(NL, NN, np.log10(flops_mlp_parallel))
ax.set_xlabel('Number of hidden layers')
ax.set_ylabel('Number of neurons')
ax.set_zlabel('Log_10 (FLOPS)')
ax.legend(['Single frequency-dep GFDN', 'Sum of parallel GFDNs'])
ax.view_init(elev=10, azim=-45)  # Elevation (vertical), Azimuth (horizontal)

fig.savefig('figures/compare_flops_mlp.png')
plt.show()